In [2]:
import pandas as pd
UserProfile=pd.read_csv('weiboUserProfile_csv.txt',header=None,sep=':')
UserProfile.columns=['User_id','Place','Gender','Year','Horoscope']
UserProfile.head()

C:\Users\admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,User_id,Place,Gender,Year,Horoscope
0,1381078220,Shanghai Putuo District,female,NaN,NaN
1,1723807154,"Xuhui District, Shanghai",male,NaN,Leo
2,1500165530,Shanghai Pudong New Area,female,1989,NaN
3,1246687181,Shanghai Pudong New Area,female,NaN,Aquarius
4,1871921945,Shanghai,female,1982,NaN


In [3]:
UserInfo=UserProfile.count(axis=0, level=None, numeric_only=False)
print(UserInfo)

User_id      507471
Place        319704
Gender       319704
Year         185577
Horoscope     81552
dtype: int64


In [4]:
LocationProfile=pd.read_csv('poiInfo_jiepang.txt',header=None,sep=',')
LocationProfile.columns=['Location_id','Latitude','Longitude','Place','Comment_1','Comment_2']
LocationProfile.head()

,Location_id,Latitude,Longitude,Place,Comment_1,Comment_2
0,j:1A4F169FAF4A8E,31.240782,121.548460,shanghai,Leisure / Entertainment / Culture,Cinema
1,j:229AF0E87884E5D7E9B827A442E07E4B,22.297639,114.178022,NaN,NaN,NaN
2,j:15805000759218470558,22.297428,114.172607,overseas,Public institution/office/residential,Apartment / Community / Lane
3,j:2551510332343224024,-33.815418,151.003494,overseas,NaN,NaN
4,j:494CC18DC9A57D3B1770D5560DC6681F,23.900358,120.587242,changhua,Public institution/office/residential,Office/office


In [5]:
LocationInfo=LocationProfile.count(axis=0, level=None, numeric_only=False)
print(LocationInfo)

Location_id    1016370
Latitude        963874
Longitude       963874
Place           981502
Comment_1       490784
Comment_2       490784
dtype: int64


In [1]:
import pandas as pd
CheckinInfo=pd.read_csv('checkin_bj_uniq.txt',header=None,sep='\t')
CheckinInfo.columns=['User_id','Time','latitude','longititude','Item_id']

In [2]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

a1=CheckinInfo.groupby( [ "User_id", "Item_id"] ).size().to_frame(name = 'count').reset_index()

#Creating Item_label
b_item=sorted(a1["Item_id"].unique())
Item_label=pd.DataFrame(b_item)
Item_label.columns=["Item_id"]

person_c = CategoricalDtype(sorted(a1.User_id.unique()), ordered=True)
thing_c = CategoricalDtype(sorted(a1.Item_id.unique()), ordered=True)

row = a1.User_id.astype(person_c).cat.codes
col = a1.Item_id.astype(thing_c).cat.codes
sparse_checkin = csr_matrix((a1['count'], (row, col)),shape=(person_c.categories.size, thing_c.categories.size))

#Converting sparse-matrix to dataframe 
#dfs = pd.SparseDataFrame(sparse_checkin, index=person_c.categories, columns=thing_c.categories,default_fill_value=0)

In [3]:
import numpy as np

# zero to zero mapping
# X.data[:] = np.sqrt(X.data)<-- non-zero values
# X.A<-- overall array

#Use sparse[:,:].toarray() to print elemnts

print("No of check-ins:",np.count_nonzero(sparse_checkin.data))
print("Users * Items:",sparse_checkin.shape)
a,b=sparse_checkin.shape
density=np.count_nonzero(sparse_checkin.data)/(a*b)
sparsity=1-density
print("Sparsity",sparsity)

No of check-ins: 1992299
Users * Items: (55650, 213673)
Sparsity 0.9998324518253869


In [9]:
#creating binary_visit matrix
binary_checkin=sparse_checkin>0
binary_checkin=1*binary_checkin


In [10]:
#already generated..dont execute again.
#to write the pyTable
#creating confidence matrix
import tables as tb
import tqdm
with tb.open_file('confid_matrix.h5', 'w') as f:
    filters = tb.Filters(complevel=5, complib='blosc')
    out = f.create_carray(f.root, 'data', tb.Float32Atom(), shape=sparse_checkin.shape, filters=filters)
    alpha=3
    with tqdm.tqdm(total=55650) as progress:
        for i in range(55650):
            out[i,]=(alpha*sparse_checkin[i,].toarray()) + 1
            progress.update(1)
    f.close()
#to read the pyTable
# with tb.open_file('confid_matrix.h5', 'r') as f:
#     a = f.root.data
#     for i in range(100):
#         row = a[i,:] #only one row gets loaded into memory
#         print(row)

In [11]:
import implicit
from implicit.als import AlternatingLeastSquares
import numpy as np

In [12]:
model_class=AlternatingLeastSquares
params = {'factors': 16, 'dtype': np.float32}
model=model_class(**params)
model.approximate_similar_items = False

In [13]:
from implicit.nearest_neighbours import bm25_weight
sparse_checkin_2 = bm25_weight(sparse_checkin, K1=100, B=0.8)

In [14]:
import time
print("training model :als")
start = time.time()
model.fit(sparse_checkin_2.T.tocsr())
print("trained model in %0.2fs" %(time.time() - start))

training model :als


100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:10<00:00,  1.47it/s]


trained model in 10.75s


In [15]:
user=list(sorted(a1.User_id.unique()))
item=list(sorted(a1.Item_id.unique()))

import tqdm
import codecs

start = time.time()
output_filename = 'recommend_implicit.txt'
user_item = sparse_checkin_2.tocsr()

with tqdm.tqdm(total=len(user)) as progress:
    with open(output_filename, "w") as o:
        for u_id in range(len(user)):
            for i_id, score in model.recommend(u_id, user_item):
                o.write("%s\t%s\t%s\n" %(user[u_id], item[i_id], score))
            progress.update(1)           
print("generated recommendations in %0.2fs" %(time.time() - start))

100%|███████████████████████████████████████████████████████████████████████████| 55650/55650 [04:18<00:00, 215.34it/s]


generated recommendations in 258.45s


In [24]:
test=CheckinInfo.set_index("User_id",drop=False)

In [16]:
#TODO:  Evaluation of recommender
#https://stats.stackexchange.com/questions/226825/what-metric-should-i-use-for-assessing-implicit-matrix-factorization-recommender
#https://github.com/statisticianinstilettos/recmetrics
    
#TODO:  Metadata Embeddings for User and Item Cold-start Recommendations
#https://towardsdatascience.com/creating-a-hybrid-content-collaborative-movie-recommender-using-deep-learning-cc8b431618af
#https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243
#https://stackoverflow.com/questions/33757974/how-to-add-user-and-item-metadata-in-recommendation-engine-and-which-python-open
#https://github.com/jalajthanaki/Movie_recommendation_engine

#TODO:matchbox and SVD feature box
#https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/score-matchbox-recommender
#https://github.com/Azure-Readiness/hol-azure-machine-learning/blob/master/008-lab-recommendation-system.md
#https://github.com/beechung/Latent-Factor-Models
#https://github.com/Coder-Yu/RecQ


#recommender
#https://www.coursera.org/specializations/recommender-systems

#Matrix Factorization
#https://medium.com/@connectwithghosh/simple-matrix-factorization-example-on-the-movielens-dataset-using-pyspark-9b7e3f567536
#https://lazyprogrammer.me/tutorial-on-collaborative-filtering-and-matrix-factorization-in-python/
#http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/

# TODO NOW:
# https://github.com/lyst/lightfm
    # https://github.com/jalajthanaki/Movie_recommendation_engine
#https://github.com/NicolasHug/Surprise
#https://github.com/DefuLian/recsys


In [6]:
import sys
!{sys.executable} -m pip install lightfm

  Using cached https://files.pythonhosted.org/packages/e9/8e/5485ac5a8616abe1c673d1e033e2f232b4319ab95424b42499fabff2257f/lightfm-1.15.tar.gz
  Complete output from command C:\Users\admin\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\admin\\AppData\\Local\\Temp\\pip-install-406033zc\\lightfm\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\admin\AppData\Local\Temp\pip-wheel-jcz6xp4i --python-tag cp37:
  Compiling without OpenMP support.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\lightfm
  copying lightfm\cross_validation.py -> build\lib.win-amd64-3.7\lightfm
  copying lightfm\data.py -> build\lib.win-amd64-3.7\lightfm
  copying lightfm\evaluation.py -> build\lib.win-amd64-3.7\lightfm
  copying lightfm\lightfm.py -> build\lib.win-amd64-3.7\ligh

  Failed building wheel for lightfm
Command "C:\Users\admin\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\admin\\AppData\\Local\\Temp\\pip-install-406033zc\\lightfm\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\admin\AppData\Local\Temp\pip-record-5fx8lfmt\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\admin\AppData\Local\Temp\pip-install-406033zc\lightfm\


In [5]:
import sys
!{sys.executable} -m pip install --upgrade setuptools

  Found existing installation: setuptools 40.2.0
    Uninstalling setuptools-40.2.0:
      Successfully uninstalled setuptools-40.2.0


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.


In [21]:
user=list(sorted(a1.User_id.unique()))
item=list(sorted(a1.Item_id.unique()))


In [22]:
user[0]

35277